In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# all_runs

- run locations

In [3]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
current run,..\..\..\..
2050 (lower WFH and no telecommute),\\modelstation1\c$\workspace\sc_2050_base_year...
"2050 (lower WFH, no telecommute, new LU)",\\modelstation1\c$\workspace\sc_2050_base_year...


## System Summary

In [4]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')

In [14]:

network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
#Light rail boardings
lr_boardings = summary_data.load_agg_data('transit/light_rail_boardings.csv')
# drop total rows from light rail boardings
lr_boardings = lr_boardings[lr_boardings['Unnamed: 0']!='Total']
lr_ridership = lr_boardings.groupby('source')['modeled_5to20'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [17]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    'Light Rail Boardings': lr_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
source,,,,,,,
2050 (lower WFH and no telecommute),"100,959,145","3,258,070","425,539","1,515,011","512,199",4.7%,"35,687"
"2050 (lower WFH, no telecommute, new LU)","95,144,481","2,966,375","318,427","1,298,849","417,074",4.2%,"33,812"
current run,"82,956,230","2,553,675","237,452","483,956","135,620",2.1%,"41,715"


In [7]:
# os.getcwd()

In [8]:
# # Populations near HCT and frequent transit
# df_time = pd.read_csv(r'../../../../outputs/agg/dash/trip_time_total.csv')
# df_ff = pd.read_csv(r'../../../../outputs/agg/dash/trip_sov_ff_time.csv')
# hh = pd.read_csv(r'../../../../outputs/agg/dash/hh_transit_dist.csv')
# person = pd.read_csv(r'../../../../outputs/agg/dash/person_transit_type.csv')

## Land Use

In [18]:
# Land Use Summary
df_parcels = summary_data.load_landuse('landuse/parcels_urbansim.txt')

In [19]:
# Summarize households and total jobs
df_pivot = pd.pivot_table(df_parcels, index='source', values=['hh_p','emptot_p'], aggfunc='sum')
df_pivot.rename(columns={'hh_p': 'Households', 'emptot_p':'Jobs'}, inplace=True)
# df_pivot = df_pivot.style.format('{:,.f}')
df_pivot[['Households','Jobs']].style.format('{:,.0f}')

,Households,Jobs
source,,
2050 (lower WFH and no telecommute),"2,422,603","3,137,221"
"2050 (lower WFH, no telecommute, new LU)","2,421,059","3,161,923"
current run,"1,736,129","2,190,946"


In [20]:
# Summarize jobs by sector
# df.columns

#### Jobs by Sector

In [21]:
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
                                'empofc_p', 'empoth_p', 'empret_p', 'emprsc_p', 'empsvc_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'empedu_p': 'Education', 'empfoo_p':'Food', 'empgov_p':'Government',
                         'empind_p':'Industrial', 'empmed_p':'Medical', 'empofc_p':'Office',
                         'empoth_p':'Other', 'empret_p':'Retail', 'emprsc_p':'Resource', 'empsvc_p':'Service'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Education,Food,Government,Industrial,Medical,Office,Other,Retail,Resource,Service
source,,,,,,,,,,
2050 (lower WFH and no telecommute),"282,038","269,830","171,282","367,503","358,855","924,772","162,475","379,308",0,"221,158"
"2050 (lower WFH, no telecommute, new LU)","287,539","270,522","179,539","369,405","359,833","923,858","163,738","386,158",0,"221,331"
current run,"167,143","157,285","222,537","341,587","242,463","582,273","155,306","176,184",0,"146,168"


#### Student Enrollment

In [22]:
# Students by grade level
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['stugrd_p', 'stuhgh_p', 'stuuni_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'stugrd_p': 'Grade School', 'stuhgh_p':'High School', 'stuuni_p':'College'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Grade School,High School,College
source,,,
2050 (lower WFH and no telecommute),"517,319","206,601","224,710"
"2050 (lower WFH, no telecommute, new LU)","442,413","211,261","136,681"
current run,"442,413","211,261","136,681"


In [24]:
# Join parcel lookup data to parcel files to do some county and district summaries
# Load parcel geography lookup from soundcast db
import sqlite3
conn = sqlite3.connect(r'../../../../inputs/db/soundcast_inputs_2023.db')
df_geog_lookup = pd.read_sql_query("SELECT ParcelID, CountyName, district_name FROM parcel_2023_geography", conn)

In [25]:
# Merge lookup data to parcels
df_parcels = df_parcels.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

#### Total Employment by Distict

In [26]:
# Group total employment by county and source
df_pivot = pd.pivot_table(df_parcels, index='CountyName', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,2050 (lower WFH and no telecommute),"2050 (lower WFH, no telecommute, new LU)",current run
CountyName,,,
King,"2,013,406","2,045,960","1,442,506"
Kitsap,"138,181","147,942","92,707"
Outside Region,549,0,0
Pierce,"517,345","473,614","334,336"
Snohomish,"467,740","494,407","321,397"


#### Total Employment by Distict

In [28]:
# Group total employment by district and source
df_pivot = pd.pivot_table(df_parcels, index='district_name', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,2050 (lower WFH and no telecommute),"2050 (lower WFH, no telecommute, new LU)",current run
district_name,,,
East Side,"688,421","615,476","420,109"
Everett-Lynwood-Edmonds,"125,250","295,998","158,328"
Kitsap,"138,182","147,942","92,707"
North Seattle-Shoreline,"290,757","190,173","147,826"
Renton-FedWay-Kent,"622,509","531,467","343,930"
S.Kitsap,"96,530","23,537","19,990"
Seattle CBD,"165,235","509,268","394,252"
South Pierce,"307,246","220,779","167,286"
Suburban Snohomish,"342,490","198,412","163,069"


## Synthetic Population

In [29]:
def count_by_county(output_path: str, summary_var: str, expfac_name: str, axis_name: str):
    df = summary_data.load_agg_data(output_path)

    tab = df.groupby(['source',summary_var])[expfac_name].sum().\
        unstack(summary_var).\
        rename_axis(columns={summary_var: axis_name})
    tab['Total'] = tab.sum(axis=1)

    display(tab.style.format('{:,.0f}'))

In [30]:
count_by_county('agg/dash/pptyp_county.csv','person_county','psexpfac','Population by county')

Population by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
2050 (lower WFH and no telecommute),"3,071,847","356,124",281,"1,311,285","1,087,666","5,827,203"
"2050 (lower WFH, no telecommute, new LU)","2,988,607","354,233",14,"1,237,744","1,207,563","5,788,161"
current run,"2,303,851","275,120",8,"925,555","851,485","4,356,019"


In [31]:
count_by_county('agg/dash/hh_geog.csv','hh_county','hhexpfac','Households by county')

Households by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
2050 (lower WFH and no telecommute),"1,121,117","42,324","255,972","225,160","1,644,573"
"2050 (lower WFH, no telecommute, new LU)","1,184,801","60,986","308,856","298,332","1,852,975"
current run,"866,673","39,114","196,913","187,282","1,289,982"


## employment by work location

In [32]:
count_by_county('agg/dash/person_worker_type.csv','person_work_county','psexpfac','Employment by county')

Employment by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
2050 (lower WFH and no telecommute),"1,723,378","114,376",458,"418,024","397,336","2,653,572"
"2050 (lower WFH, no telecommute, new LU)","1,216,757","115,252",nan,"407,318","455,182","2,194,509"
current run,"1,287,769","84,566",nan,"298,838","281,683","1,952,856"


## Parking Costs

In [48]:
# Daily parking cost totals (ppricdyp)
pd.pivot_table(df_parcels, index='source', values='ppricdyp', aggfunc='sum').style.format('{:,.0f}')

,ppricdyp
source,
2050 (lower WFH and no telecommute),"100,944,431"
"2050 (lower WFH, no telecommute, new LU)","102,112,889"
current run,"1,029,600"


In [ ]:
# # Parking costs by district
# pd.pivot_table(df_parcels, index='district_name', columns='source', values='pprichrp', aggfunc='sum').style.format('{:,.0f}')

source,2050 (lower WFH and no telecommute),"2050 (lower WFH, no telecommute, new LU)",current run
district_name,,,
East Side,"6,874,931","3,235,660","17,700"
Everett-Lynwood-Edmonds,"835,171","1,292,854","1,400"
Kitsap,"704,201","702,056","5,800"
North Seattle-Shoreline,"3,289,225","5,857,784","14,900"
Renton-FedWay-Kent,"6,773,264","5,802,658",0
S.Kitsap,"285,793",0,0
Seattle CBD,"1,751,674","5,856,685","145,700"
South Pierce,"2,966,534","908,757",0
Suburban Snohomish,"2,025,296","2,151,457",0


In [ ]:
# df_buffered.columns

Index(['parcelid', 'xcoord_p', 'ycoord_p', 'sqft_p', 'taz_p', 'lutype_p',
       'hh_p', 'stugrd_p', 'stuhgh_p', 'stuuni_p', 'empedu_p', 'empfoo_p',
       'empgov_p', 'empind_p', 'empmed_p', 'empofc_p', 'empret_p', 'empsvc_p',
       'empoth_p', 'emptot_p', 'parkdy_p', 'parkhr_p', 'ppricdyp', 'pprichrp',
       'hh_1', 'stugrd_1', 'stuhgh_1', 'stuuni_1', 'empedu_1', 'empfoo_1',
       'empgov_1', 'empind_1', 'empmed_1', 'empofc_1', 'empret_1', 'empsvc_1',
       'empoth_1', 'emptot_1', 'parkdy_1', 'parkhr_1', 'ppricdy1', 'pprichr1',
       'nodes1_1', 'nodes3_1', 'nodes4_1', 'tstops_1', 'nparks_1', 'aparks_1',
       'hh_2', 'stugrd_2', 'stuhgh_2', 'stuuni_2', 'empedu_2', 'empfoo_2',
       'empgov_2', 'empind_2', 'empmed_2', 'empofc_2', 'empret_2', 'empsvc_2',
       'empoth_2', 'emptot_2', 'parkdy_2', 'parkhr_2', 'ppricdy2', 'pprichr2',
       'nodes1_2', 'nodes3_2', 'nodes4_2', 'tstops_2', 'nparks_2', 'aparks_2',
       'dist_lbus', 'dist_ebus', 'dist_crt', 'dist_fry', 'dist_lrt',


In [73]:
# # Check buffered parcels distance to light rail
# df_buffered = summary_data.load_landuse('landuse/buffered_parcels.txt')

# # Merge geography to file
# df_buffered = df_buffered.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

In [74]:

# df_buffered.groupby(['source','district_name'])['dist_lbus'].describe()
# # df_buffered.columns